In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Define the path to your file. Make sure it is the correct path
# Ensure the file 'DIAGNOSES_ICD.csv' exists in this location in your Google Drive.
file_path = '/content/drive/MyDrive/CorteX/data/MIMIC/'

# Load the data
diagnosis_data = pd.read_csv(file_path+'DIAGNOSES_ICD.csv')
diagnosis_data_code = pd.read_csv(file_path+'D_ICD_DIAGNOSES.csv')

In [3]:
# How many patients do we have
print(diagnosis_data['subject_id'].nunique())

100


In [4]:
#Finding most common diseases in dataset
top_5 = diagnosis_data['icd9_code'].value_counts().head(5)
print(top_5)
# Map top_5 icd9-code to the corresponding short name in diagnosis_data_code with the same icd9-code
for i in range(top_5.shape[0]):
    top_5.iloc[i] = diagnosis_data_code[diagnosis_data_code['icd9_code'] == top_5.index[i]]['long_title'].iloc[0]

print(top_5)

icd9_code
4019     53
42731    48
5849     45
4280     39
25000    31
Name: count, dtype: int64
icd9_code
4019                    Unspecified essential hypertension
42731                                  Atrial fibrillation
5849                     Acute kidney failure, unspecified
4280                 Congestive heart failure, unspecified
25000    Diabetes mellitus without mention of complicat...
Name: count, dtype: object


<ipython-input-4-ebd79f18d23d>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unspecified essential hypertension' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  top_5.iloc[i] = diagnosis_data_code[diagnosis_data_code['icd9_code'] == top_5.index[i]]['long_title'].iloc[0]


In [5]:
# Creating our massive patient dataset
patients_data = pd.read_csv(file_path+'PATIENTS.csv')
bigPatientData = patients_data.copy()

################################################### Getting icd9 codes and diagnoses #####################################################
# Merge bigPatientData with the diagnoses data to get icd9_codes
merged_data = bigPatientData.merge(
    diagnosis_data[['subject_id', 'icd9_code']],
    on='subject_id',
    how='left'
)

# Now merge with the long title data, selecting only necessary columns
merged_data = merged_data.merge(
    diagnosis_data_code[['icd9_code', 'long_title']],
    on='icd9_code',
    how='left'
)

# Group by subject_id and aggregate the diagnoses into lists
diagnoses_aggregated = merged_data.groupby('subject_id').agg({
    'icd9_code': lambda x: [str(code) for code in x.dropna().unique()],
    'long_title': lambda x: list(x.dropna())
}).reset_index()

# Rename the columns
diagnoses_aggregated.columns = ['subject_id', 'icd9_codes', 'diagnoses (long_titles)']

# Merge the aggregated diagnoses back into bigPatientData
bigPatientData = bigPatientData.merge(diagnoses_aggregated, on='subject_id', how='left')

# Ensure to drop any duplicate columns
bigPatientData = bigPatientData.loc[:, ~bigPatientData.columns.duplicated()]

In [6]:
############################################# Adding Age ##################################################################
bigPatientData['age'] = np.random.normal(loc=39.8, scale=10, size=len(bigPatientData))

In [7]:
######################################## Adding Perscriptions ######################################################################
drugs_data = pd.read_csv(file_path+'PRESCRIPTIONS.csv')

# Converting all three drug columns to lowercase
drugs_data['drug'] = drugs_data['drug'].str.lower()
drugs_data['drug_name_poe'] = drugs_data['drug_name_poe'].str.lower()
drugs_data['drug_name_generic'] = drugs_data['drug_name_generic'].str.lower()

# Append to bigPatientData columns that takes all the drugs a patient is taking from drugs_data and adds it as a list

bigPatientData['prescriptions'] = bigPatientData['subject_id'].apply(
    lambda x: list(drugs_data[drugs_data['subject_id'] == x]['drug'].dropna().unique())
)

bigPatientData['prescriptions_poe'] = bigPatientData['subject_id'].apply(
    lambda x: list(drugs_data[drugs_data['subject_id'] == x]['drug_name_poe'].dropna().unique())
)

bigPatientData['prescriptions_generic'] = bigPatientData['subject_id'].apply(
    lambda x: list(drugs_data[drugs_data['subject_id'] == x]['drug_name_generic'].dropna().unique())
)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,icd9_codes,diagnoses (long_titles),age,prescriptions,prescriptions_poe,prescriptions_generic
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1,"[99591, 99662, 5672, 40391, 42731, 4280, 4241,...","[Sepsis, Infection and inflammatory reaction d...",41.626021,"[sodium chloride 0.9% flush, glipizide, metop...","[sodium chloride 0.9% flush, glipizide, metop...","[sodium chloride 0.9% flush, glipizide, metop..."
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1,"[570, 07030, 07054, 30401, 2875, 2760]","[Acute and subacute necrosis of liver, Viral h...",35.000979,[],[],[]
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1,"[0389, 41071, 78551, 486, 42731, 20280, 4240, ...","[Unspecified septicemia, Subendocardial infarc...",41.212574,"[furosemide, azithromycin, acetaminophen, nitr...","[furosemide, azithromycin, acetaminophen, nitr...","[furosemide, azithromycin, acetaminophen, nitr..."
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1,"[81201, 4928, 8028, 8024, 99812, 41511, 2851, ...","[Closed fracture of surgical neck of humerus, ...",48.730862,"[hydromorphone, sodium chloride 0.9% flush, n...","[hydromorphone, sodium chloride 0.9% flush, n...","[hydromorphone, sodium chloride 0.9% flush, n..."
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1,"[0389, 51881, 5770, 30390, 5781, 5845, 2848, 5...","[Unspecified septicemia, Acute respiratory fai...",16.934976,"[propofol, heparin flush cvl (100 units/ml), ...","[propofol, heparin flush cvl (100 units/ml), ...","[propofol, heparin flush, phytonadione, albumi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,31838,44083,M,2057-11-15 00:00:00,2114-02-20 00:00:00,2114-02-20 00:00:00,2114-02-20 00:00:00,1,"[1510, 5185, 4019, 2724, 5119, 4239, 2859, V10...","[Malignant neoplasm of cardia, Unspecified ess...",38.946365,"[diphenhydramine, sodium chloride 0.9% flush,...","[diphenhydramine, sodium chloride 0.9% flush,...","[diphenhydramine, sodium chloride 0.9% flush,..."
96,31853,44154,M,1878-05-14 00:00:00,2178-05-15 00:00:00,2178-05-15 00:00:00,2178-05-15 00:00:00,1,"[0389, 5185, 5070, 99592, 42731, 72400, 36250,...","[Unspecified septicemia, Pneumonitis due to in...",44.689180,"[pneumococcal vac polyvalent, sodium chloride ...","[pneumococcal vac polyvalent, sodium chloride ...","[pneumococcal vac polyvalent, sodium chloride ..."
97,31867,44212,F,2078-06-16 00:00:00,2124-01-29 00:00:00,NaN,2124-01-29 00:00:00,1,"[03811, 78552, 5845, 48241, 51881, 5990, 3481,...",[Methicillin susceptible Staphylococcus aureus...,34.334150,"[sodium chloride 0.9% flush, chlorhexidine gl...","[sodium chloride 0.9% flush, chlorhexidine gl...","[sodium chloride 0.9% flush, chlorhexidine gl..."
98,31870,44222,M,2107-06-27 00:00:00,2182-08-03 00:00:00,2182-08-03 00:00:00,NaN,1,"[42781, 5849, E9429, 41401, 40390, 5853, 25060...","[Sinoatrial node dysfunction, Acute kidney fai...",45.706136,"[bisacodyl, senna, docusate sodium (liquid), h...","[bisacodyl, senna, docusate sodium (liquid), h...","[bisacodyl, bisacodyl (rectal), senna, docusat..."


In [11]:
# Get most common prescriptions
drugs_data
top_15_drugs = drugs_data['drug'].value_counts().head(15)
print(top_15_drugs)

drug
potassium chloride             529
d5w                            439
0.9% sodium chloride           409
ns                             362
furosemide                     346
insulin                        300
iso-osmotic dextrose           265
5% dextrose                    256
sw                             244
magnesium sulfate              206
morphine sulfate               206
sodium chloride 0.9%  flush    206
acetaminophen                  156
heparin                        145
calcium gluconate              144
Name: count, dtype: int64


In [13]:
##################################### Random Name Generation #########################################
import random
# Sample lists of first and last names
first_names = ['John', 'Jane', 'Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Kelsey', 'Addison', 'Sam']
last_names = ['Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson','Franklin', 'Pierce']

# Assuming bigPatientData is already defined
num_patients = bigPatientData.shape[0]

# Generate random first and last names and append to bigPatientData
bigPatientData['first_name'] = [random.choice(first_names) for _ in range(num_patients)]
bigPatientData['last_name'] = [random.choice(last_names) for _ in range(num_patients)]

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,icd9_codes,diagnoses (long_titles),age,prescriptions,prescriptions_poe,prescriptions_generic,first_name,last_name
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1,"[99591, 99662, 5672, 40391, 42731, 4280, 4241,...","[Sepsis, Infection and inflammatory reaction d...",41.626021,"[sodium chloride 0.9% flush, glipizide, metop...","[sodium chloride 0.9% flush, glipizide, metop...","[sodium chloride 0.9% flush, glipizide, metop...",Eva,Jones
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1,"[570, 07030, 07054, 30401, 2875, 2760]","[Acute and subacute necrosis of liver, Viral h...",35.000979,[],[],[],Sam,Smith
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1,"[0389, 41071, 78551, 486, 42731, 20280, 4240, ...","[Unspecified septicemia, Subendocardial infarc...",41.212574,"[furosemide, azithromycin, acetaminophen, nitr...","[furosemide, azithromycin, acetaminophen, nitr...","[furosemide, azithromycin, acetaminophen, nitr...",Eva,Franklin
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1,"[81201, 4928, 8028, 8024, 99812, 41511, 2851, ...","[Closed fracture of surgical neck of humerus, ...",48.730862,"[hydromorphone, sodium chloride 0.9% flush, n...","[hydromorphone, sodium chloride 0.9% flush, n...","[hydromorphone, sodium chloride 0.9% flush, n...",Frank,Pierce
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1,"[0389, 51881, 5770, 30390, 5781, 5845, 2848, 5...","[Unspecified septicemia, Acute respiratory fai...",16.934976,"[propofol, heparin flush cvl (100 units/ml), ...","[propofol, heparin flush cvl (100 units/ml), ...","[propofol, heparin flush, phytonadione, albumi...",Jane,Jones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,31838,44083,M,2057-11-15 00:00:00,2114-02-20 00:00:00,2114-02-20 00:00:00,2114-02-20 00:00:00,1,"[1510, 5185, 4019, 2724, 5119, 4239, 2859, V10...","[Malignant neoplasm of cardia, Unspecified ess...",38.946365,"[diphenhydramine, sodium chloride 0.9% flush,...","[diphenhydramine, sodium chloride 0.9% flush,...","[diphenhydramine, sodium chloride 0.9% flush,...",John,Wilson
96,31853,44154,M,1878-05-14 00:00:00,2178-05-15 00:00:00,2178-05-15 00:00:00,2178-05-15 00:00:00,1,"[0389, 5185, 5070, 99592, 42731, 72400, 36250,...","[Unspecified septicemia, Pneumonitis due to in...",44.689180,"[pneumococcal vac polyvalent, sodium chloride ...","[pneumococcal vac polyvalent, sodium chloride ...","[pneumococcal vac polyvalent, sodium chloride ...",Charlie,Williams
97,31867,44212,F,2078-06-16 00:00:00,2124-01-29 00:00:00,NaN,2124-01-29 00:00:00,1,"[03811, 78552, 5845, 48241, 51881, 5990, 3481,...",[Methicillin susceptible Staphylococcus aureus...,34.334150,"[sodium chloride 0.9% flush, chlorhexidine gl...","[sodium chloride 0.9% flush, chlorhexidine gl...","[sodium chloride 0.9% flush, chlorhexidine gl...",Eva,Franklin
98,31870,44222,M,2107-06-27 00:00:00,2182-08-03 00:00:00,2182-08-03 00:00:00,NaN,1,"[42781, 5849, E9429, 41401, 40390, 5853, 25060...","[Sinoatrial node dysfunction, Acute kidney fai...",45.706136,"[bisacodyl, senna, docusate sodium (liquid), h...","[bisacodyl, senna, docusate sodium (liquid), h...","[bisacodyl, bisacodyl (rectal), senna, docusat...",Jane,Pierce


In [15]:
# Create csv of patient data
bigPatientData.to_csv(file_path+'bigPatientData.csv', index=False)